In [65]:
import re
import numpy as np
import pandas as pd
import jieba
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics.pairwise import cosine_similarity
import pickle
from tqdm import tqdm
from pprint import pprint
import os

# 加载停用词
with open('chinese_stopwords.txt','r', encoding='utf-8') as file:
    stopwords=[i[:-1] for i in file.readlines()]
#stopwords = [line.strip() for line in open('chinsesstoptxt.txt',encoding='UTF-8').readlines()]
# 数据加载
news = pd.read_csv('sqlResult.csv',encoding='gb18030')
print(news.shape)
print(news.head(5))

# 处理缺失值
print('------------------------')
print(news[news.content.isnull()].head(5))


news=news.dropna(subset=['content'])
print(news.shape)


(89611, 7)
      id      author                  source  \
0  89617         NaN  快科技@http://www.kkj.cn/   
1  89616         NaN  快科技@http://www.kkj.cn/   
2  89615         NaN  快科技@http://www.kkj.cn/   
3  89614         NaN                     新华社   
4  89613  胡淑丽_MN7479                   深圳大件事   

                                             content  \
0  此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...   
1  骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...   
2  此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...   
3    这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n   
4  （原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...   

                                             feature  \
0  {"type":"科技","site":"cnbeta","commentNum":"37"...   
1  {"type":"科技","site":"cnbeta","commentNum":"15"...   
2  {"type":"科技","site":"cnbeta","commentNum":"18"...   
3  {"type":"国际新闻","site":"环球","commentNum":"0","j...   
4  {"type":"新闻","site":"网易热门","commentNum":"978",...   

                          

In [66]:
# 分词
def split_text(text):
    #return ' '.join([w for w in list(jieba.cut(re.sub('\s|[%s]' % (punctuation),'',text))) if w not in stopwords])
    text = text.replace(' ', '')
    text = text.replace('\n', '')
    text2 = jieba.cut(text.strip())
    result = ' '.join([w for w in text2 if w not in stopwords])
    return result

print(news.iloc[0].content)
print(split_text(news.iloc[0].content))

此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/体验版内测，稳定版暂不受影响），以确保工程师可以集中全部精力进行系统优化工作。有人猜测这也是将精力主要用到MIUI 9的研发之中。
MIUI 8去年5月发布，距今已有一年有余，也是时候更新换代了。
当然，关于MIUI 9的确切信息，我们还是等待官方消息。

 当然 MIUI9 确切 信息 等待 官方消息


In [67]:
if not os.path.exists("corpus.pkl"):
    # 对所有文本进行分词
    corpus=list(map(split_text,[str(i) for i in news.content]))
    print(corpus[0])
    print(len(corpus))
    print(corpus[1])
    # 保存到文件，方便下次调用
    with open('corpus.pkl','wb') as file:
        pickle.dump(corpus, file)
else:
    # 调用上次处理的结果
    with open('corpus.pkl','rb') as file:
        corpus = pickle.load(file)
        print(corpus[0])
        print(len(corpus))
        print(corpus[1])

 当然 MIUI9 确切 信息 等待 官方消息
87054
 当然 高通 骁龙 未来 也许 见到 三星 Exynos 联发科 华为 麒麟 小米 澎湃 进入 Windows10 桌面 平台


In [77]:
# 得到corpus的TF-IDF矩阵
countvectorizer = CountVectorizer(encoding='gb18030',min_df=0.015)
tfidftransformer = TfidfTransformer()

countvector = countvectorizer.fit_transform(corpus)
print(countvector.shape)

tfidf = tfidftransformer.fit_transform(countvector)
print(tfidf)

(87054, 884)
  (0, 590)	0.237736608488
  (0, 461)	0.287807002707
  (0, 271)	0.431560057041
  (0, 416)	0.263569509136
  (0, 432)	0.217387276604
  (0, 669)	0.267150299769
  (0, 860)	0.267413014466
  (0, 184)	0.246785583728
  (0, 822)	0.150818545995
  (0, 385)	0.185018693496
  (0, 103)	0.201078563673
  (0, 667)	0.283109136374
  (0, 263)	0.229198530311
  (0, 44)	0.262255632828
  (0, 174)	0.234573238002
  (1, 399)	0.494772117943
  (1, 424)	0.186910393549
  (1, 67)	0.180080374995
  (1, 721)	0.195238625458
  (1, 579)	0.18803588095
  (1, 468)	0.206490112785
  (1, 50)	0.179296930749
  (1, 525)	0.394227649957
  (1, 474)	0.151293813494
  (1, 391)	0.138511916522
  :	:
  (87053, 817)	0.230148976937
  (87053, 598)	0.344413356651
  (87053, 571)	0.0559908759501
  (87053, 641)	0.0610848278538
  (87053, 684)	0.0626017820772
  (87053, 42)	0.0536046870433
  (87053, 294)	0.0533525802612
  (87053, 58)	0.0605567014517
  (87053, 533)	0.0517811906986
  (87053, 649)	0.0630373058684
  (87053, 46)	0.0599096415093

In [69]:
# 标记是否为自己的新闻
label=list(map(lambda source: 1 if '新华' in str(source) else 0,news.source))
#print(label)

# 数据集切分
X_train, X_test, y_train, y_test = train_test_split(tfidf.toarray(), label, test_size = 0.3, random_state=42)
clf = MultinomialNB()
clf.fit(X=X_train, y=y_train)

"""
# 进行CV=3折交叉验证
scores=cross_validate(clf, X_train, y_train, scoring=('accuracy','precision','recall','f1'), cv=3, return_train_score=True)
pprint(scores)
"""
y_predict = clf.predict(X_test)
def show_test_reslt(y_true,y_pred):
    print('accuracy:',accuracy_score(y_true,y_pred))
    print('precison:',precision_score(y_true,y_pred))
    print('recall:',recall_score(y_true,y_pred))
    print('f1_score:',f1_score(y_true,y_pred))

show_test_reslt(y_test, y_predict)

accuracy: 0.889726997741
precison: 0.962384953982
recall: 0.914101102242
f1_score: 0.937621832359


In [88]:
# 使用模型检测抄袭新闻
prediction = clf.predict(tfidf.toarray())
labels = np.array(label)
# compare_news_index中有两列：prediction为预测，labels为真实值
compare_news_index = pd.DataFrame({'prediction':prediction,'labels':labels})
# copy_news_index：可能是Copy的新闻（即找到预测为1，但是实际不是“新华社”）
copy_news_index=compare_news_index[(compare_news_index['prediction'] == 1) & (compare_news_index['labels'] == 0)].index
# 实际为新华社的新闻
xinhuashe_news_index=compare_news_index[(compare_news_index['labels'] == 1)].index
print('可能为Copy的新闻条数：', copy_news_index)

可能为Copy的新闻条数： Int64Index([   4,   24,   28,   30,   32,   35,   37,   38,   43,   45,
            ...
            8538, 8539, 8543, 8544, 8546, 8548, 8549, 8551, 8552, 8555],
           dtype='int64', length=2818)


In [76]:
#文章聚类后保存列表
if not os.path.exists("label.pkl"):
    # 使用k-means对文章进行聚类
    from sklearn.preprocessing import Normalizer
    from sklearn.cluster import KMeans
    normalizer = Normalizer()
    scaled_array = normalizer.fit_transform(tfidf.toarray())

    # 使用K-Means, 对全量文档进行聚类
    kmeans = KMeans(n_clusters=25,random_state=42,n_jobs=-1)
    k_labels = kmeans.fit_predict(scaled_array)
    # 保存到文件，方便下次调用
    with open('label.pkl','wb') as file:
        pickle.dump(k_labels, file)
    print(k_labels.shape)
    print(k_labels[0])
else:
    # 调用上次处理的结果
    with open('label.pkl','rb') as file:
        k_labels = pickle.load(file)
        print(k_labels.shape)
        print(k_labels)

(87054,)
[16 16 16 ..., 16 16 23]


In [78]:
#将类别对应编号保存列表
if not os.path.exists("id_class.pkl"):
    # 创建id_class
    id_class = {index:class_ for index, class_ in enumerate(k_labels)}
    # 保存到文件，方便下次调用
    with open('id_class.pkl','wb') as file:
        pickle.dump(id_class, file)
else:
    # 调用上次处理的结果
    with open('id_class.pkl','rb') as file:
        id_class = pickle.load(file)
        print(id_class)

{0: 16, 1: 16, 2: 16, 3: 0, 4: 16, 5: 8, 6: 16, 7: 8, 8: 16, 9: 8, 10: 8, 11: 8, 12: 16, 13: 8, 14: 8, 15: 8, 16: 16, 17: 16, 18: 8, 19: 8, 20: 12, 21: 8, 22: 16, 23: 8, 24: 16, 25: 16, 26: 16, 27: 16, 28: 16, 29: 8, 30: 16, 31: 16, 32: 16, 33: 16, 34: 16, 35: 16, 36: 16, 37: 16, 38: 16, 39: 0, 40: 16, 41: 16, 42: 14, 43: 16, 44: 16, 45: 17, 46: 8, 47: 8, 48: 16, 49: 16, 50: 16, 51: 0, 52: 14, 53: 16, 54: 16, 55: 16, 56: 16, 57: 14, 58: 14, 59: 16, 60: 16, 61: 16, 62: 14, 63: 10, 64: 17, 65: 14, 66: 17, 67: 14, 68: 14, 69: 16, 70: 16, 71: 17, 72: 16, 73: 16, 74: 16, 75: 16, 76: 16, 77: 16, 78: 16, 79: 16, 80: 16, 81: 16, 82: 16, 83: 16, 84: 16, 85: 16, 86: 14, 87: 16, 88: 8, 89: 14, 90: 14, 91: 16, 92: 16, 93: 16, 94: 14, 95: 14, 96: 17, 97: 16, 98: 8, 99: 14, 100: 8, 101: 16, 102: 16, 103: 16, 104: 16, 105: 8, 106: 14, 107: 16, 108: 16, 109: 14, 110: 8, 111: 14, 112: 14, 113: 14, 114: 16, 115: 8, 116: 14, 117: 14, 118: 14, 119: 14, 120: 16, 121: 23, 122: 16, 123: 16, 124: 16, 125: 23,

In [79]:
#按类别排序属于新华社的文章列表
if not os.path.exists("class_id.pkl"):
    from collections import defaultdict
    # 创建你class_id字段，key为classId,value为文档index
    class_id = defaultdict(set)
    for index,class_ in id_class.items():
        # 只统计新华社发布的class_id
        if index in xinhuashe_news_index.tolist():
            class_id[class_].add(index)
    # 保存到文件，方便下次调用
    with open('class_id.pkl','wb') as file:
        pickle.dump(class_id, file)
else:
    # 调用上次处理的结果
    with open('class_id.pkl','rb') as file:
        class_id = pickle.load(file)
        print(class_id[0])

{3, 65544, 32785, 32786, 65553, 65566, 65572, 65573, 65574, 39, 65575, 51, 65587, 65588, 65589, 65591, 65593, 32828, 32830, 32831, 65619, 32853, 32854, 32855, 32856, 32857, 32858, 65632, 32866, 65635, 65636, 65637, 65638, 32871, 32878, 65648, 65649, 65650, 65651, 65663, 65665, 65666, 65669, 65670, 65671, 65673, 65674, 65676, 65678, 65679, 65682, 65696, 65698, 65699, 65700, 65701, 65702, 78187, 65706, 32950, 32951, 32953, 65723, 65725, 65726, 65727, 65728, 65730, 32966, 32968, 32995, 32996, 32997, 33002, 33003, 33004, 33005, 65774, 33006, 33007, 33008, 33022, 33023, 33097, 33098, 33099, 65867, 52172, 52178, 52179, 52180, 52181, 52182, 65936, 52183, 52184, 65946, 65947, 65948, 33203, 33204, 33209, 33210, 33211, 33225, 33226, 33227, 33228, 66018, 66019, 66020, 66031, 66041, 33283, 33284, 66070, 66071, 66072, 33339, 33340, 33346, 33348, 587, 33369, 33370, 33371, 33372, 66146, 66147, 66148, 66150, 66157, 33402, 33404, 33405, 33406, 33407, 33408, 33409, 66196, 33435, 669, 33438, 33441, 33444

In [73]:
# 输出每个类别的 文档个数
count=0
for k in class_id:
    print(count, len(class_id[k]))
    count +=1

# 查找相似文本（使用聚类结果进行filter）
def find_similar_text(cpindex, top=10):
    # 只在新华社发布的文章中查找
    dist_dict={i:cosine_similarity(tfidf[cpindex],tfidf[i]) for i in class_id[id_class[cpindex]]}
    # 从大到小进行排序
    return sorted(dist_dict.items(),key=lambda x:x[1][0], reverse=True)[:top]



0 12324
1 10360
2 6919
3 3789
4 3506
5 1851
6 1653
7 1762
8 728
9 2280
10 949
11 1732
12 1603
13 1474
14 1564
15 9749
16 1843
17 874
18 2124
19 2008
20 1358
21 2844
22 2800
23 1254
24 1507


In [101]:
import editdistance
# 指定某篇文章的相似度
#print(copy_news_index)
cpindex = 8555 # 在copy_news_index
print('是否在新华社', cpindex in xinhuashe_news_index)
print('是否在copy_news', cpindex in copy_news_index)
print('8555', 8555 in xinhuashe_news_index)
#print('3134是否在copy_news', 3134 in copy_news_index)

#print(cpindex)
similar_list = find_similar_text(cpindex)
print(similar_list)
print('怀疑抄袭:\n', news.iloc[cpindex].content)
# 找一篇相似的原文
similar2 = similar_list[0][0]
print('相似原文:\n', news.iloc[similar2].content)
# 求任意两篇文章的编辑距离 
print('编辑距离:',editdistance.eval(corpus[cpindex], corpus[similar2]))


是否在新华社 False
是否在copy_news True
8555 False
[(68206, array([[ 0.48793322]])), (30033, array([[ 0.4733099]])), (27846, array([[ 0.43609555]])), (13261, array([[ 0.40351228]])), (37245, array([[ 0.38944651]])), (59076, array([[ 0.36957268]])), (85759, array([[ 0.36537502]])), (67235, array([[ 0.36444518]])), (58165, array([[ 0.36302362]])), (33554, array([[ 0.36233523]]))]
怀疑抄袭:
 据新华社北京6月12日电 经中央军委主席习近平批准，我军新设立“八一勋章”，并组织开展首次评选。新设立的“八一勋章”，是由中央军委决定、中央军委主席签发证书并颁授的军队最高荣誉，授予在维护国家主权、安全、发展利益，推进国防和军队现代化建设中建立卓越功勋的军队人员。经过解放军、武警部队和公安现役部队提名推荐，近日产生初步候选人，向军内外公示接受评议。
此次评选颁授工作，重点选拔在促进战斗力生成提高、完成作战等重大任务、推进科技兴军、推动国防和军队现代化建设中作出杰出贡献、建立卓越功勋的英模典型。各级把组织评选作为深入推进政治建军、改革强军、依法治军，强化练兵备战的重要载体，采取提名推荐、资格审查、组织公示、全面考核、呈报审批的步骤组织实施，使评选过程成为深入学习英模事迹、激励官兵昂扬奋进的过程。
“八一勋章”是军人最崇高的荣誉，英雄模范是民族最闪亮的坐标。广大官兵积极参与推荐评选，表示要以习主席关怀厚爱为强大动力，以“八一勋章”评选颁授为重要契机，学英雄、建功绩、当先进，踊跃投身强军兴军伟大实践，以优异成绩迎接党的十九大胜利召开。（相关报道见4版）

相似原文:
 　　新华社上海5月22日电中共中央政治局委员、中央军委副主席范长龙近日在驻湖北、江苏、上海部队和院校调研时强调，全军和武警部队要坚决贯彻习主席重要决策指示，坚定自觉地维护核心、听从指挥，扎实推进改革任务，狠抓年度工作落实，以优异成绩迎接党的十九大胜利召开。\n中央

In [102]:
def find_similar_sentence(candidate, raw):
    similist = []
    cl = candidate.strip().split('。')
    ra = raw.strip().split('。')
    for c in cl:
        for r in ra:
            similist.append([c,r,editdistance.eval(c,r)])
    # 最相似的5个句子
    sort=sorted(similist,key=lambda x:x[2])[:5]
    for c,r,ed in sort:
        if c!='' and r!='':
            print('怀疑抄袭句:{0}\n相似原句:{1}\n 编辑距离:{2}\n'.format(c,r,ed))
# 查找copy文章 和第一相似的原文的比对
find_similar_sentence(news.iloc[cpindex].content, news.iloc[similar2].content)


怀疑抄袭句:（相关报道见4版）
相似原句:（完）\n\n
 编辑距离:8

怀疑抄袭句:（相关报道见4版）
相似原句:\n中央军委委员、空军司令员马晓天参加有关调研活动
 编辑距离:25

怀疑抄袭句:（相关报道见4版）
相似原句:要强化练兵备战，坚持战斗力标准，扎扎实实提高部队战斗力
 编辑距离:27

怀疑抄袭句:
“八一勋章”是军人最崇高的荣誉，英雄模范是民族最闪亮的坐标
相似原句:\n中央军委委员、空军司令员马晓天参加有关调研活动
 编辑距离:30

怀疑抄袭句:
“八一勋章”是军人最崇高的荣誉，英雄模范是民族最闪亮的坐标
相似原句:要强化练兵备战，坚持战斗力标准，扎扎实实提高部队战斗力
 编辑距离:30

